In [13]:
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt


In [22]:
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None  # Used for linking nodes of the same item

In [23]:
class FPTree:
    def __init__(self, transactions, min_support):
        self.min_support = min_support
        self.root = FPNode(None, 0, None)
        self.header_table = defaultdict(lambda: None)
        self.build_tree(transactions)


In [24]:
def count_frequencies(transactions):
    item_counts = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1
    return item_counts

# Example Transactions
data =transactions = [
    ["milk", "bread", "butter"],
    ["bread", "butter", "jam"],
    ["milk", "bread", "butter", "jam"],
    ["bread", "butter"],
    ["milk", "bread"]
]


# Count Frequencies
item_counts = count_frequencies(data)
print("Item Frequencies:", item_counts)


Item Frequencies: defaultdict(<class 'int'>, {'milk': 3, 'bread': 5, 'butter': 4, 'jam': 2})


#  Filter Infrequent Items

In [37]:
min_support = 2
freq_items = {item: count for item, count in item_counts.items() if count >= min_support}
print("Frequent Items:", freq_items)

Frequent Items: {'milk': 3, 'bread': 5, 'butter': 4, 'jam': 2}


# Order Transactions Based on Frequency

In [26]:
def order_transactions(transactions, freq_items):
    ordered_transactions = []
    for transaction in transactions:
        ordered = sorted([item for item in transaction if item in freq_items], key=lambda x: (-freq_items[x], x))
        ordered_transactions.append(ordered)
    return ordered_transactions

ordered_data = order_transactions(data, freq_items)
print("Ordered Transactions:", ordered_data)

Ordered Transactions: [['bread', 'butter', 'milk'], ['bread', 'butter', 'jam'], ['bread', 'butter', 'milk', 'jam'], ['bread', 'butter'], ['bread', 'milk']]


In [27]:
def insert_tree(transaction, node, header_table):
    if not transaction:
        return
    first_item = transaction[0]
    if first_item in node.children:
        node.children[first_item].count += 1
    else:
        node.children[first_item] = FPNode(first_item, 1, node)
        # Link header table
        if header_table[first_item] is None:
            header_table[first_item] = node.children[first_item]
        else:
            current = header_table[first_item]
            while current.link is not None:
                current = current.link
            current.link = node.children[first_item]
    
    # Recursively insert remaining items
    insert_tree(transaction[1:], node.children[first_item], header_table)


In [29]:
header_table = defaultdict(lambda: None)
root = FPNode(None, 0, None)

for transaction in ordered_data:
    insert_tree(transaction, root, header_table)
print("FP-Tree Construction Complete")

FP-Tree Construction Complete


In [35]:
def build_graph(node, G, labels):
    if node.item is not None:
        labels[node.item] = f"{node.item} ({node.count})"
    for child in node.children.values():
        G.add_edge(node.item if node.item else "Root", child.item)
        build_graph(child, G, labels)

def visualize_tree(root):
    G = nx.DiGraph()
    labels = {}
    build_graph(root, G, labels)
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, labels=labels, node_size=2000, node_color="lightblue")
    plt.show()

In [34]:
def print_tree(node, indent=0):
    if node.item is not None:
        print('  ' * indent + f"{node.item} ({node.count})")
    for child in node.children.values():
        print_tree(child, indent + 1)


print("FP-Tree Structure:")
print_tree(root)

FP-Tree Structure:
  bread (5)
    butter (4)
      milk (2)
        jam (1)
      jam (1)
    milk (1)
